# 使用SAM2对图像进行分割标注
相比上一版,将SAM预训练模型更改为SAM2,SAM2相比较SAM1,将模型从SVM更改为Transformer,提高了泛化能力和微调效果.

**参考文献:**
- [SAM2 Paper](https://ai.meta.com/research/publications/sam-2-segment-anything-in-images-and-videos/)
- [SAM2 - GitHub](https://github.com/facebookresearch/segment-anything-2)
- [(Official)SAM2 Image Predictor Example notebook - GitHub](https://github.com/facebookresearch/segment-anything-2/blob/main/notebooks/image_predictor_example.ipynb)
- [(Official)Automatic Mask Generator Example notebook - GitHub](https://github.com/facebookresearch/segment-anything-2/blob/main/notebooks/automatic_mask_generator_example.ipynb)

## SVM2主要优势有:
- 提升了预训练模型的微调效果.
- 除了支持视频处理外,图像分割的准确度和速度都比SAM1有所提升.
- 可以将一组图片视为同一个序列来处理,可能可以解决单张图片不能输入全部信息的情况.

## 预计下一步:
1. 参考其他开放数据集的图像和数据标注方式,设计一个自己标注数据的方法
2. 设计根据模型预测masks找出多边形顶点的方法
    - 可能使用opencv库或者别的小规模机器学习模型识别,因为masks中像素点可能不完全连续,需要测试后确定
    - 尝试是否可以让模型把每个缺陷部分都拆分成单独的masks组,之后再找顶点坐标,提升并发效率
    - 是否需要按面积/具体边数标记缺陷
3. 使用其他开放数据集微调模型,之后与原来的预训练模型效果比较,估测较好结果可能需要的数据量
4. 

## 预计使用的其他类型开放数据集
部分来源(有一组表面缺陷识别的开放数据集和paper): [GitHub - Surface Defect Detection: Dataset & Papers](https://github.com/Charmve/Surface-Defect-Detection)
1. 医学超声
2. 钢材缺陷 [Kaggle - Severstal: Steel Defect Detection](https://www.kaggle.com/c/severstal-steel-defect-detection/data)
3. 

导入所需库及最大的预训练模型

In [2]:
from numbers import Number

import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from torch.utils.data import DataLoader
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

# 选择运行设备
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

checkpoint = "./checkpoints/sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"
predictor = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))


using device: cuda


定义过程中的输出函数

In [3]:
np.random.seed(3)


def show_mask(mask, ax, random_color=False, borders=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        # Try to smooth contours
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2)
    ax.imshow(mask_image)


def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels == 1]
    neg_points = coords[labels == 0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white',
               linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white',
               linewidth=1.25)


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))


def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # boxes
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i + 1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()


def find_polygon_boundary(matrix: np.ndarray):
    """
    从二维布尔矩阵(如图形masks)中找到多边形顶点
    Args:
        matrix (np.ndarray): 单个2D numpy数组,True代表包含于多边形中.
    Returns:
        以元组形式存储单个多边形顶点的坐标,将所有顶点以列表形式返回
    """
    # 设定矩阵数据类型
    matrix = np.asarray(matrix, dtype=bool)
    # 创建空矩阵
    boundary = np.zeros_like(matrix, dtype=bool)

    pass

    boundary_list = []

    return boundary_list


class ChipDefectDatasetLoader(torch.utils.data.Dataset):
    """
    数据加载器类
    """
    def __init__(self, image_paths: list[str], mask_paths: list[str]) -> None:  # transform是转换函数
        self.image_paths = image_paths
        self.mask_paths = mask_paths

    def __len__(self) -> Number:
        return len(self.image_paths)

    def __getitem__(self, idx: int):
        image = cv2.imread(self.image_paths[idx])
        mask = cv2.imread(self.mask_paths[idx])
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask
        

NameError: name 'Callable' is not defined

In [ ]:
# 加载训练数据
image_paths = ["./dataset/images"]
mask_paths = ["./dataset/masks"]
train_dataset = ChipDefectDatasetLoader(image_paths, mask_paths)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
# 定义优化器和损失函数
optimizer = torch.optim.Adam(predictor.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# 开始训练
for epoch in range(num_epochs):
    sam_predictor.train()
    running_loss = 0.0
    for images, masks in train_loader:
        optimizer.zero_grad()

        # 模型前向传播
        masks_pred = predictor(images)

        # 计算损失
        loss = criterion(masks_pred, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')